In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import joblib

In [2]:
final_rmse = 5000
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [3]:
def prepare_data(df, sequence_length):
    data = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler

In [4]:
def build_gru_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(GRU(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(GRU(units, return_sequences=True))
    model.add(GRU(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [5]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    
    global final_rmse
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'GRUmodel_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    gru_loss_history = loss_history.history['loss']
    
    gru_predictions = model.predict(X_test)
    gru_predictions = scaler.inverse_transform(gru_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, gru_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    if rmse <= final_rmse:
        final_rmse = rmse
        joblib.dump(model, "BTC_model_gru.pkl")
        plt.plot(y_test, label='Actual')
        plt.plot(gru_predictions, label='Predicted')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        plt.savefig(f'BTC_actual_vs_predicted_GRU.png')
        plt.close()
    
        plt.figure(figsize=(10, 6))
        plt.plot(loss_history.history['loss'], label='Training Loss')
        plt.plot(loss_history.history['val_loss'], label='Validation Loss')
        plt.title('Epoch Loss Curve')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f'BTC_loss_curve_GRU.png')
        plt.close()
    
    return model, rmse


In [6]:
symbol = 'BTC-USD'
start_date = '2020-01-01'
end_date = '2024-01-31'
epochs_range = [100]
batch_sizes = [4,8]
sequence_lengths = [25,20]
units_range = [50, 100]
layers_range = [2, 3]
results = []

In [7]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [8]:
for epochs in epochs_range:
    for batch_size in batch_sizes:
        for sequence_length in sequence_lengths:
            for units in units_range:
                for layers in layers_range:
                    print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
                    X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
                    model = build_gru_model(X_train, units=units, layers=layers)
                    model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)
                    results.append({'epochs': epochs, 'batch_size': batch_size, 'sequence_length': sequence_length, 'units': units, 'layers': layers, 'rmse': rmse})

Training model with epochs=100, batch_size=4, sequence_length=25, units=50, layers=2


Epoch 1/100

268/268 [==============================] - 22s 41ms/step - loss: 0.0040 - val_loss: 0.0010
Epoch 2/100
  1/268 [..............................] - ETA: 16s - loss: 0.0018

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 8s 31ms/step - loss: 0.0015 - val_loss: 4.2353e-04
Epoch 3/100
268/268 [==============================] - 9s 32ms/step - loss: 7.9498e-04 - val_loss: 1.8995e-04
Epoch 4/100
268/268 [==============================] - 8s 30ms/step - loss: 8.1301e-04 - val_loss: 2.1477e-04
Epoch 5/100
268/268 [==============================] - 8s 29ms/step - loss: 6.9932e-04 - val_loss: 4.7180e-04
Epoch 6/100
268/268 [==============================] - 8s 29ms/step - loss: 8.4371e-04 - val_loss: 1.8559e-04
Epoch 7/100
268/268 [==============================] - 8s 29ms/step - loss: 7.4894e-04 - val_loss: 2.9932e-04
Epoch 8/100
268/268 [==============================] - 9s 33ms/step - loss: 9.4549e-04 - val_loss: 2.2185e-04
Epoch 9/100
268/268 [==============================] - 9s 33ms/step - loss: 5.6605e-04 - val_loss: 1.3921e-04
Epoch 10/100
268/268 [==============================] - 8s 30ms/step - loss: 7.3662e-04 - val_loss: 1.3414e-04
Epoch 11/100
268/268 [===

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 11s 42ms/step - loss: 0.0015 - val_loss: 5.0593e-04
Epoch 3/100
268/268 [==============================] - 11s 40ms/step - loss: 9.7270e-04 - val_loss: 3.3324e-04
Epoch 4/100
268/268 [==============================] - 11s 40ms/step - loss: 7.6990e-04 - val_loss: 1.6082e-04
Epoch 5/100
268/268 [==============================] - 10s 39ms/step - loss: 8.9999e-04 - val_loss: 6.3959e-04
Epoch 6/100
268/268 [==============================] - 11s 41ms/step - loss: 8.2410e-04 - val_loss: 2.5152e-04
Epoch 7/100
268/268 [==============================] - 11s 40ms/step - loss: 7.8949e-04 - val_loss: 5.5319e-04
Epoch 8/100
268/268 [==============================] - 10s 39ms/step - loss: 8.7632e-04 - val_loss: 2.2216e-04
Epoch 9/100
268/268 [==============================] - 11s 39ms/step - loss: 0.0010 - val_loss: 1.7434e-04
Epoch 10/100
268/268 [==============================] - 11s 39ms/step - loss: 8.9775e-04 - val_loss: 2.2347e-04
Epoch 11/100
268/268

...

134/134 [==============================] - 5s 37ms/step - loss: 0.0018 - val_loss: 4.1540e-04
Epoch 3/100
134/134 [==============================] - 5s 37ms/step - loss: 0.0010 - val_loss: 2.6228e-04
Epoch 4/100
134/134 [==============================] - 5s 38ms/step - loss: 7.9033e-04 - val_loss: 2.1695e-04
Epoch 5/100
134/134 [==============================] - 5s 36ms/step - loss: 8.5127e-04 - val_loss: 5.6622e-04
Epoch 6/100
134/134 [==============================] - 5s 36ms/step - loss: 8.2802e-04 - val_loss: 2.8315e-04
Epoch 7/100
134/134 [==============================] - 5s 36ms/step - loss: 7.5440e-04 - val_loss: 0.0011
Epoch 8/100
134/134 [==============================] - 5s 36ms/step - loss: 7.3825e-04 - val_loss: 3.8906e-04
Epoch 9/100
134/134 [==============================] - 5s 38ms/step - loss: 6.1161e-04 - val_loss: 1.3446e-04
Epoch 10/100
134/134 [==============================] - 5s 36ms/step - loss: 6.7343e-04 - val_loss: 1.5077e-04
Epoch 11/100
134/134 [===========

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 6s 41ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 3/100
134/134 [==============================] - 6s 46ms/step - loss: 9.5787e-04 - val_loss: 2.2244e-04
Epoch 4/100
134/134 [==============================] - 6s 41ms/step - loss: 0.0012 - val_loss: 2.0397e-04
Epoch 5/100
134/134 [==============================] - 5s 40ms/step - loss: 7.4612e-04 - val_loss: 2.3379e-04
Epoch 6/100
134/134 [==============================] - 6s 41ms/step - loss: 7.6277e-04 - val_loss: 1.3251e-04
Epoch 7/100
134/134 [==============================] - 5s 39ms/step - loss: 5.7740e-04 - val_loss: 1.4099e-04
Epoch 8/100
134/134 [==============================] - 5s 39ms/step - loss: 8.6896e-04 - val_loss: 3.1072e-04
Epoch 9/100
134/134 [==============================] - 6s 41ms/step - loss: 6.4231e-04 - val_loss: 1.8691e-04
Epoch 10/100
134/134 [==============================] - 6s 41ms/step - loss: 7.8438e-04 - val_loss: 1.3228e-04
Epoch 11/100
134/134 [===========

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 6s 48ms/step - loss: 0.0014 - val_loss: 0.0030
Epoch 3/100
134/134 [==============================] - 6s 47ms/step - loss: 0.0012 - val_loss: 2.5343e-04
Epoch 4/100
134/134 [==============================] - 7s 55ms/step - loss: 9.1880e-04 - val_loss: 2.6289e-04
Epoch 5/100
134/134 [==============================] - 7s 50ms/step - loss: 8.8936e-04 - val_loss: 2.2825e-04
Epoch 6/100
134/134 [==============================] - 7s 50ms/step - loss: 6.6288e-04 - val_loss: 1.8019e-04
Epoch 7/100
134/134 [==============================] - 7s 50ms/step - loss: 6.7138e-04 - val_loss: 5.0215e-04
Epoch 8/100
134/134 [==============================] - 6s 48ms/step - loss: 7.7198e-04 - val_loss: 5.0566e-04
Epoch 9/100
134/134 [==============================] - 6s 48ms/step - loss: 8.6236e-04 - val_loss: 1.6366e-04
Epoch 10/100
134/134 [==============================] - 6s 47ms/step - loss: 0.0011 - val_loss: 3.6125e-04
Epoch 11/100
134/134 [===============

In [9]:
for result in results:
    print(result)

{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 832.456450107897}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 3, 'rmse': 748.4604107033904}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 2, 'rmse': 751.0444186266182}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 3, 'rmse': 744.9672838049554}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 50, 'layers': 2, 'rmse': 739.0995196881445}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 50, 'layers': 3, 'rmse': 738.0897380165186}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 100, 'layers': 2, 'rmse': 788.2587605921552}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 100, 'layers': 3, 'rmse': 797.4305103443014}
{'epochs': 100, 'batch_size': 8, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 827.3478302313071}
{'epochs': 100, 

In [11]:
df1 = yf.download("BTC-USD", "2020-01-01", "2024-01-31")

[*********************100%%**********************]  1 of 1 completed


In [12]:
df1 = df1["Close"]

In [13]:
df1.max()

67566.828125

In [14]:
df1.min()

4970.7880859375